# Code

## CUDA Utilities

In [1]:
%%writefile cuda_stuff.cuh
#ifndef cuda_stuff_H
#define cuda_stuff_H

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

//MACRO TO DEBUG CUDA FUNCTIONS
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

void device_synchronize();

#endif


Writing cuda_stuff.cuh


In [2]:
%%writefile cuda_stuff.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

#include "cuda_stuff.cuh"

void device_synchronize(){
    gpuErrchk(cudaDeviceSynchronize());
}

Writing cuda_stuff.cu


## Matrix Tools

In [3]:
%%writefile fmatrix.cuh
#ifndef fmatrices_H
#define fmatrices_H
#include <stddef.h>

typedef struct {
    float* data;
    size_t cols;
    size_t rows;
} fmatrix;

/* transform matrix index to vector offset
   Since CUDA uses column major,
   nb_rows = number of rows */
#define IDX2C(i,j,nb_rows) (((j)*(nb_rows))+(i))

/* Access element (i,j) of matrix mat */
#define getfm(mat,i,j) (mat.data[IDX2C(i,j,mat.rows)])


size_t fmatrix_elements(fmatrix mat);
size_t fmatrix_size(fmatrix mat);
void fmatrix_init(fmatrix mat, float f);
/** Assert that the matrix is coherent: all fields nonzero. */
void fmatrix_assert();

fmatrix fmatrix_create_on_host(size_t rows, size_t cols);
fmatrix fmatrix_create_on_device(size_t rows, size_t cols);

void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device);
void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device);

void fmatrix_free_on_host(fmatrix* mat);
void fmatrix_free_on_device(fmatrix* mat);

/** Print the first nb rows of the matrix mat
 *  on the host.
 *  If nb<0, print all rows.
 */
void fmatrix_host_print(fmatrix mat, int nb=-1);

/** Print the first nb rows of the matrix mat
 *  on the device.
 *  If nb<0, print all rows.
 */
void fmatrix_device_print(fmatrix mat, int nb=-1);

#endif


Writing fmatrix.cuh


In [4]:
%%writefile fmatrix.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

#include "cuda_stuff.cuh"
#include "fmatrix.cuh"

size_t fmatrix_elements(fmatrix mat) {
     return mat.cols*mat.rows;
}

size_t fmatrix_size(fmatrix mat) {
     return fmatrix_elements(mat) * sizeof(float);
}

void fmatrix_init(fmatrix mat, float f) {
    for (int i = 0; i < mat.rows; i++){
        for (int j = 0; j < mat.cols; j++){
            mat.data[IDX2C(i,j,mat.rows)] = f;
    }
  }
}

void fmatrix_assert(fmatrix mat) {
    assert(mat.data);
    assert(mat.cols);
    assert(mat.rows);
}



fmatrix fmatrix_create_on_host(size_t rows, size_t cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    mat.data = (float*)malloc(fmatrix_size(mat));
    assert(mat.data);
    return mat;
}

fmatrix fmatrix_create_on_device(size_t rows, size_t cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    gpuErrchk(
        cudaMalloc((void **)&(mat.data), fmatrix_size(mat))
    );
    return mat;
}

void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_device.data, mat_host.data,
                   fmatrix_size(mat_host),
                   cudaMemcpyHostToDevice
                   )
        );
}

void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_host.data, mat_device.data,
                   fmatrix_size(mat_device),
                   cudaMemcpyDeviceToHost
                   )
        );
}

void fmatrix_free_on_host(fmatrix* mat) {
    fmatrix_assert(*mat);
  free(mat->data);
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

void fmatrix_free_on_device(fmatrix* mat) {
    fmatrix_assert(*mat);
  gpuErrchk(cudaFree(mat->data));
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

void fmatrix_host_print(fmatrix mat, int nb){
    if (nb<0 || nb > mat.rows) {
        nb = mat.rows;
    }
    printf("[\n");
    for (int i = 0 ; i < nb; i++){
      for (int j = 0 ; j<mat.cols; j++){
        printf("%f", getfm(mat,i,j));
        if (j+1<mat.cols) {
          printf(",\t");
        }
      }
      if (i+1<nb) {
        printf(";\n");
      }
    }
    if (nb < mat.rows) {
      printf("\n...\n");
    }
  printf("\n]\n");
}

void fmatrix_device_print(fmatrix mat, int nb){
   // allocate copy
   fmatrix tmp = fmatrix_create_on_host(mat.rows, mat.cols);
   fmatrix_data_to_host(tmp, mat);
   fmatrix_host_print(tmp,nb);
   fmatrix_free_on_host(&tmp);
}



Writing fmatrix.cu


## Matrix Math

In [5]:
%%writefile sgemm.cuh
#ifndef sgemm_H
#define sgemm_H

#include <string>
#include "fmatrix.cuh"

void mat_mul(fmatrix A, fmatrix B, fmatrix C, std::string arg);

#endif

Writing sgemm.cuh


In [16]:
%%writefile sgemm.cu
#include <stdio.h>
#include <stdlib.h>
#include <string>
#include <time.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include "cublas_v2.h"

#include "cuda_stuff.cuh"
#include "sgemm.cuh"
#include "fmatrix.cuh"

#define THREADS_PER_BLOCK 1024
#define TILE_WIDTH 32

using namespace std;

static cublasHandle_t handle;
static int cublas_init = 0;

/* basic matrix multiplication C = alpha*A*B + beta*C on host as reference for the speedup */
void matrixMultiplication_basic_host(float alpha, fmatrix A, fmatrix B, float beta, fmatrix C) {
  float tmp = 0;
  for (int i = 0; i<A.rows; i++){
    for (int j = 0; j<B.cols; j++){
      for (int k = 0; k<A.cols; k++){
        tmp += alpha * getfm(A,i, k) * getfm(B, k, j);
      }
      getfm(C, i, j) = beta * getfm(C, i, j) + tmp;
      tmp = 0;
    }
  }
}

/* TODO : 3 different versions of matrix multiplication C = alpha*A*B + beta*C on device */
__global__
void matmul_basic_kernel(float alpha, float *A, float *B, float beta, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB) {
  int row = threadIdx.y + blockIdx.y * blockDim.y;
  int col = threadIdx.x + blockIdx.x * blockDim.x;
  if (row < nb_LigneA && col < nb_ColB){
    float tmp = 0.0f;
    for (int k = 0; k < nb_LigneB; k++){
      tmp += alpha * A[IDX2C(row, k, nb_LigneA)] * B[IDX2C(col, k, nb_LigneB)]; /* Calcul et stockage du résultat du produit entre A et B */
    }
    C[IDX2C(row, col, nb_LigneA)] = beta * C[IDX2C(row, col, nb_LigneA)] + tmp; /* Calcul final en ajoutant la valeur de C et mise à jour de la matrice résultat */
  }
}

void matrixMultiplication_basic(float alpha, fmatrix d_A, fmatrix d_B, float beta, fmatrix d_C) {
  dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
  dim3 dimGrid((d_A.rows + TILE_WIDTH - 1)/TILE_WIDTH, (d_B.cols + TILE_WIDTH - 1)/TILE_WIDTH);

  matmul_basic_kernel <<< dimGrid, dimBlock >>> (alpha, d_A.data, d_B.data, beta, d_C.data, d_A.cols, d_B.cols, d_A.rows, d_B.rows);

}

/**********************/
__global__
void matmul_tiled_kernel(float alpha, float *A, float *B, float beta, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB){
  __shared__ float tileA[TILE_WIDTH][TILE_WIDTH];
  __shared__ float tileB[TILE_WIDTH][TILE_WIDTH];

  int row = blockIdx.y * TILE_WIDTH + threadIdx.y;
  int col = blockIdx.x * TILE_WIDTH + threadIdx.x;

  float sum = 0.0f;

  for (int t = 0; t < (nb_ColA + TILE_WIDTH - 1) / TILE_WIDTH; ++t) {
      if (row < nb_LigneA && t * TILE_WIDTH + threadIdx.x < nb_ColA)
          tileA[threadIdx.y][threadIdx.x] = A[row * nb_ColA + t * TILE_WIDTH + threadIdx.x];
      else
          tileA[threadIdx.y][threadIdx.x] = 0.0f;

      if (col < nb_ColB && t * TILE_WIDTH + threadIdx.y < nb_ColA)
          tileB[threadIdx.y][threadIdx.x] = B[(t * TILE_WIDTH + threadIdx.y) * nb_ColB + col];
      else
          tileB[threadIdx.y][threadIdx.x] = 0.0f;

      __syncthreads();

      for (int k = 0; k < TILE_WIDTH; ++k) {
          sum += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];
      }

      __syncthreads();
  }

  if (row < nb_LigneA && col < nb_ColB)
      C[row * nb_ColB + col] = alpha * sum + beta * C[row * nb_ColB + col];
}


void matrixMultiplication_tiled(float alpha, fmatrix d_A, fmatrix d_B, float beta, fmatrix d_C){
  dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
  dim3 dimGrid((d_A.rows + TILE_WIDTH - 1) / TILE_WIDTH, (d_B.cols + TILE_WIDTH - 1) / TILE_WIDTH);

  matmul_tiled_kernel <<< dimGrid, dimBlock >>> (alpha, d_A.data, d_B.data, beta, d_C.data, d_A.cols, d_B.cols, d_A.rows, d_B.rows);
}

/**********************/
void matrixMultiplication_cublas(float alpha, fmatrix d_A, fmatrix d_B, float beta, fmatrix d_C){
  cublasHandle_t handle;
  cublasCreate(&handle);

  cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N,
              d_C.cols, d_C.rows, d_A.cols, // M, N, K
              &alpha,
              d_B.data, d_B.cols, // B (K x N)
              d_A.data, d_A.cols, // A (M x K)
              &beta,
              d_C.data, d_C.cols); // C (M x N)

  cublasDestroy(handle);
}




/*MAIN SGEMM*/
void gen_mat_mul(float alpha, fmatrix A, fmatrix B, float beta, fmatrix C, std::string arg){
    if (arg == "cpu"){
        matrixMultiplication_basic_host(alpha, A, B, beta, C);
    } else {
      if (arg == "gpu_basic"){
          matrixMultiplication_basic(alpha, A, B, beta, C);

      } else if (arg == "gpu_tiled"){
          matrixMultiplication_tiled(alpha, A, B, beta, C);

      } else if (arg == "gpu_cublas"){
         matrixMultiplication_cublas(alpha, A, B, beta, C);

      } else{
          printf("Matrix Multiplication argument is Wrong");
          exit(0);
      }
      // wait for everything to finish
      device_synchronize();
    }
}

void mat_mul(fmatrix A, fmatrix B, fmatrix C, std::string arg){
 gen_mat_mul(1.0, A, B, 0.0, C, arg);
}


Overwriting sgemm.cu


# Main

In [12]:
%%writefile main.cu

#include <stdio.h>
#include <stdlib.h>
#include "fmatrix.cuh"
#include "sgemm.cuh"

#define TILE_WIDTH 32
#define SIZE 40

int main(void)
{
  /* Allocate and initialize data on host */
  fmatrix A = fmatrix_create_on_host(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix_init(A, 1.0);
  fmatrix B = fmatrix_create_on_host(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix_init(B, 2.0);
  fmatrix C = fmatrix_create_on_host(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix_init(C, 0.0);

  /* Allocate data on device */
  fmatrix d_A = fmatrix_create_on_device(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix d_B = fmatrix_create_on_device(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix d_C = fmatrix_create_on_device(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);

  /* Transfer A and B on device */
  fmatrix_data_to_device(A, d_A);
  fmatrix_data_to_device(B, d_B);
  fmatrix_data_to_device(C, d_C);

  clock_t start, end;
  float cpu_time_used;

  /* Start calculation "cpu", "gpu_basic", "gpu_tiled", "gpu_cublas" */
  /************** "cpu" *******************/
  start = clock();
  mat_mul(A, B, C, "cpu");
  end = clock();
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("Time taken by CPU in milliseconds: %.2f\n", cpu_time_used);


  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    }
    printf("Max error: %f\n", maxError);
  }
  fmatrix_init(C, 0.0);

  /************** "gpu_basic" *******************/
  start = clock();
  mat_mul(d_A, d_B, d_C, "gpu_basic");
  end = clock();
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("GPU basic matrix multiplication in milliseconcs : %.2f\n", cpu_time_used);

  /* Retrieve the result */
  fmatrix_data_to_host(C, d_C);
  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    }
    printf("Max error: %f\n", maxError);
  }
  fmatrix_init(C, 0.0);
  fmatrix_data_to_device(C, d_C);


 /************** "gpu_tiled" *******************/
  start = clock();
  mat_mul(d_A, d_B, d_C, "gpu_tiled");
  end = clock();
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("GPU tiled matrix multiplication in milliseconcs : %.2f\n", cpu_time_used);

  /* Retrieve the result */
  fmatrix_data_to_host(C, d_C);
  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    }
    printf("Max error: %f\n", maxError);
  }
  fmatrix_init(C, 0.0);
  fmatrix_data_to_device(C, d_C);


  /************** "gpu_cublas" *******************/
  for(int warmup = 0; warmup < 5; warmup++){
    mat_mul(d_A, d_B, d_C, "gpu_cublas");
  }
  fmatrix_init(C, 0.0);
  fmatrix_data_to_device(C, d_C);

  start = clock();
  mat_mul(d_A, d_B, d_C, "gpu_cublas");
  end = clock();
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("GPU cuBLAS matrix multiplication in milliseconcs : %.2f\n", cpu_time_used);

  /* Retrieve the result */
  fmatrix_data_to_host(C, d_C);
  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    }
    printf("Max error: %f\n", maxError);
  }
  fmatrix_init(C, 0.0);
  fmatrix_data_to_device(C, d_C);

  /* Free */
  fmatrix_free_on_host(&A);
  fmatrix_free_on_host(&B);
  fmatrix_free_on_host(&C);
  fmatrix_free_on_device(&d_A);
  fmatrix_free_on_device(&d_B);
  fmatrix_free_on_device(&d_C);

  cudaError_t err;
  err = cudaPeekAtLastError();
  if (err != cudaSuccess){
      fprintf(stderr,"GPUassert: add launch failed with the error : %s \n", cudaGetErrorString(err));
      exit(err);
  }
  err = cudaDeviceSynchronize() ;
  if (err != cudaSuccess){
      fprintf(stderr,"GPUassert: add execution failed with the error : %s \n", cudaGetErrorString(err));
      exit(err);
  }
}

Overwriting main.cu


# Compiling

In [13]:
!nvcc -lcublas sgemm.cu  fmatrix.cu  cuda_stuff.cu main.cu -arch=sm_75 -o main

sgemm.cu(19): warning #177-D: variable "handle" was declared but never referenced
  static cublasHandle_t handle;
                        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

sgemm.cu(20): warning #177-D: variable "cublas_init" was declared but never referenced
  static int cublas_init = 0;
             ^



# Experiments

In [14]:
! ./main

Time taken by CPU in milliseconds: 20439.76
Max error: 0.000000
GPU basic matrix multiplication in milliseconcs : 13.97
Max error: 0.000000
GPU tiled matrix multiplication in milliseconcs : 10.34
Max error: 0.000000
GPU cuBLAS matrix multiplication in milliseconcs : 1.95
Max error: 0.000000


# <font color='LightBlue'> Analyse des résultats </font> #

Il apparaît que pour des matrices de taille 40 * 32 = 1280, on obtient les résultats suivants :

* Calcul classique sur CPU -> 18794 ms<br>
* Calcul classique sur GPU -> 14.03 ms<br>
* Calcul par tuile sur GPU -> 10.32 ms<br>
* Calcul avec cuBLAS sur GPU -> 2.15 ms

L'erreur nulle pour toutes les méthodes assure une justesse de calcul, ce qui les différencie sera donc leur temps d'exécution.

Pour des matrices de taille 10, les temps de calcul obtenus sont les suivants :

Calcul classique sur CPU : 0.03 ms<br>
Calcul classique sur GPU : 0.19 ms<br>
Calcul par tuiles sur GPU : 0.04 ms<br>
Calcul avec cuBLAS sur GPU : 0.37 ms
Ces résultats montrent qu'une taille de bloc fixe n'est pas toujours optimale, le calcul direct sur CPU étant le plus rapide dans ce cas.

Nous avons également testé une taille de matrice plus grande, 1278, qui n'est pas un multiple de 32 mais reste proche de l'exemple initial. Les performances obtenues sont :

Calcul classique sur CPU : 9999.47 ms<br>
Calcul classique sur GPU : 14.23 ms<br>
Calcul par tuiles sur GPU : 10.40 ms<br>
Calcul avec cuBLAS sur GPU : 1.94 ms
Bien que les calculs sur GPU semblent légèrement moins optimisés, les performances restent comparables. En revanche, sur CPU, la durée de calcul est presque divisée par deux lorsqu'on retire 2 lignes et 2 colonnes, ce qui marque une différence significative.

# Debugging
Compile with debugging info on the host (`-g`) and device (`-G`).


In [ ]:
!nvcc -g -G -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include -lcublas -lcusolver sgemm.cu fmatrix.cu cuda_stuff.cu main.cu

Run the debugger cuda-gdb, stopping at the first error that is detected. Shows first the call stack on the GPU, the values of local variables, then the call stack on the host (thread 1).

In [ ]:
! printf "set cuda api_failures stop\ncatch throw\nr UNIT\nbt\ninfo locals\nthread 1\nbt\n" > tmp.txt
! cat tmp.txt
! cuda-gdb -batch -x tmp.txt ./a.out